# Get owned game, sorted by playtime descending

In [1]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [33]:
playerid = "76561198287996067"

In [34]:
from dotenv import load_dotenv
import os
import requests

In [35]:
# Load environment variables from .env file
load_dotenv()

# Get the API_KEY from the environment variables
API_KEY = os.getenv('API_KEY')

In [36]:
url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=" + API_KEY + "&steamid=" + playerid + "&format=json"

response = requests.get(url)

if response.status_code != 200:
    print(f"Error: {response.status_code}")
    exit()

data = response.json()
games = data['response']['games']

In [37]:
import urllib.parse

games = sorted(games, key=lambda x: x['playtime_forever'], reverse=True)
games = list(games)[:10]

ACCESS_TOKEN = urllib.parse.quote_plus( os.getenv('ACCESS_TOKEN'))

def get_game_name(appid):
    url = "https://api.steampowered.com/ICommunityService/GetApps/v1/?access_token=" + str(ACCESS_TOKEN)

    for index, app in enumerate(appid):
        url += "&appids%5B" + str(index) + "%5D=" + str(app)

    print(url)
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []
    data = response.json()    

    if (len(data['response']) == 0):
        return []
    return [app['name'] for app in data['response']['apps']]

names = get_game_name([game['appid'] for game in games])
names

https://api.steampowered.com/ICommunityService/GetApps/v1/?access_token=eyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MDAxMV8yNUQ2RUJGQ19BMkYzNiIsICJzdWIiOiAiNzY1NjExOTgyODc5OTYwNjciLCAiYXVkIjogWyAid2ViOnN0b3JlIiBdLCAiZXhwIjogMTc0Mzg3NTIzMiwgIm5iZiI6IDE3MzUxNDcxNTMsICJpYXQiOiAxNzQzNzg3MTUzLCAianRpIjogIjAwMEZfMjYxNjBCQjhfMTEwMzAiLCAib2F0IjogMTczOTgxMDA1MSwgInJ0X2V4cCI6IDE3NTgyNzA2MzksICJwZXIiOiAwLCAiaXBfc3ViamVjdCI6ICIxMTMuMTg1Ljk0LjIzNyIsICJpcF9jb25maXJtZXIiOiAiMTEzLjE4NS45NC4yMzciIH0.oOtiFv9Y-cosI4x26stBU-GejN6e6gLAZfR8ZpFyUjaBNrP7on8YWm0Ogg4Udyhy1xJ-S_i6qCmY0renqMhSAg&appids%5B0%5D=105600&appids%5B1%5D=322330&appids%5B2%5D=301520&appids%5B3%5D=365670&appids%5B4%5D=550&appids%5B5%5D=730&appids%5B6%5D=782330&appids%5B7%5D=881100&appids%5B8%5D=1281930&appids%5B9%5D=379720


['Terraria',
 "Don't Starve Together",
 'Robocraft',
 'Blender',
 'Left 4 Dead 2',
 'Counter-Strike 2',
 'DOOM Eternal',
 'Noita',
 'tModLoader',
 'DOOM']

In [38]:
if len(names) > 0:
    print("Top 10 games played by the user:")
    for index, game in enumerate(games):
        print(f"{names[index]} - {int(game['playtime_forever']/60)} hours")

Top 10 games played by the user:
Terraria - 1192 hours
Don't Starve Together - 197 hours
Robocraft - 192 hours
Blender - 138 hours
Left 4 Dead 2 - 134 hours
Counter-Strike 2 - 118 hours
DOOM Eternal - 112 hours
Noita - 96 hours
tModLoader - 88 hours
DOOM - 77 hours
